## Testing out some code to hit the NHL API


Resources

* https://towardsdatascience.com/nhl-analytics-with-python-6390c5d3206d
* https://nhl-api-explorer.herokuapp.com/
* https://gitlab.com/dword4/nhlapi
* https://gitlab.com/dword4/nhlapi/-/blob/master/stats-api.md#game-ids
* https://github.com/dword4/nhlapi

Game IDs
* The first 4 digits identify the season of the game (ie. 2017 for the 2017-2018 season). 
* The next 2 digits give the type of game, where 01 = preseason, 02 = regular season, 03 = playoffs, 04 = all-star. 
* The final 4 digits identify the specific game number. 
* For regular season and preseason games, this ranges from 0001 to the number of games played. (1271 for seasons with 31 teams (2017 and onwards) and 1230 for seasons with 30 teams). 
* For playoff games, the 2nd digit of the specific number gives the round of the playoffs, the 3rd digit specifies the matchup, and the 4th digit specifies the game (out of 7).

In [8]:
# Load dependencies
import requests
import pickle
import pandas as pd
import json

In [9]:
# Set up the API call variables
year = '2021'
season_type = '02' 
max_game_id = 20
boxscore_url = '/boxscore'
livefeed_url = '/feed/live'
game_url = 'http://statsapi.web.nhl.com/api/v1/game/'

In [10]:
# Test hitting the boxscore and live feed endpoints, save as JSON files
def request_get_data(url, 
                    year, 
                    season_type, 
                    max_game_id, 
                    endpoint,
                    empty_list = [],
                    zfill = 4, 
                    multiple_games = True):

    # If multiple_games param is true, the get data for n number of games (uses for loop)
    if multiple_games == True:    
        for i in range(0, max_game_id):
            r = requests.get(url = str(url) + str(year) + str(season_type) + str(max_game_id).zfill(zfill) + str(endpoint))
            data = r.json()
            empty_list.append(data)

    # If multiple_games param is not true, then get data for the 1 game (no for loop)
    else:
        r = requests.get(url = str(url) + str(year) + str(season_type) + str(max_game_id).zfill(zfill) + str(endpoint))
        data = r.json()
        empty_list.append(data)

    # Rename to game_data_json
    game_data_list = empty_list

    return game_data_list

# Function that takes in a list, converts to json, then saves a copy of that json file in the local folder
def save_as_json(list, 
                 file_name = 'data'):

    # Save as json file in same folder the notebook is run (relative)
    with open(f'{file_name}.json', 'w') as f:
        json.dump(list, f)

    # Convert list to json
    j_data = json.dumps(list)
 
    
    return j_data

In [11]:
# For a single game, save json files for both the boxscore and livefeed api
single_game_id = 2

# Boxscore
boxscore_list = request_get_data(game_url, 
                                        year, 
                                        season_type, 
                                        single_game_id, 
                                        boxscore_url,
                                        multiple_games = True)
boxscore_json = save_as_json(boxscore_list, 'test_data/boxscore')

# Livefeed
livefeed_list = request_get_data(game_url, 
                                        year, 
                                        season_type, 
                                        single_game_id, 
                                        livefeed_url,
                                        multiple_games = True)
livefeed_json = save_as_json(livefeed_list, 'test_data/livefeed')

In [155]:
# Boxscore JSON
# Get the number of shots, goals and icetime by game for each skater
json_object = json.loads(boxscore_json)

# first layer = the game (number)
# second layerS

arlo_swag = []
player_id = list(json_object[0]['teams']['home']['players'].keys())

for i in player_id:
    player_data = {
        'player_id': i,
        'player_name': json_object[0]['teams']['home']['players'][i]['person']['fullName'],
        'player_pos': json_object[0]['teams']['home']['players'][i]['person']['primaryPosition']['name'],
        #'goals': json_object[0]['teams']['home']['players'][i]['stats']['skaterStats'].keys()
        #'shots': json_object[0]['teams']['home']['players'][i]['stats']['skaterStats']['shots'],
        #'toi': json_object[0]['teams']['home']['players'][i]['stats']['skaterStats']['timeOnIce']
    }
    arlo_swag.append(player_data)
        

arlo_swag

[{'player_id': 'ID8476905',
  'player_name': 'Chandler Stephenson',
  'player_pos': 'Center'},
 {'player_id': 'ID8474149',
  'player_name': 'Evgenii Dadonov',
  'player_pos': 'Right Wing'},
 {'player_id': 'ID8476539',
  'player_name': 'Jonathan Marchessault',
  'player_pos': 'Center'},
 {'player_id': 'ID8475913',
  'player_name': 'Mark Stone',
  'player_pos': 'Right Wing'},
 {'player_id': 'ID8474565',
  'player_name': 'Alex Pietrangelo',
  'player_pos': 'Defenseman'},
 {'player_id': 'ID8477447',
  'player_name': 'Shea Theodore',
  'player_pos': 'Defenseman'},
 {'player_id': 'ID8479639',
  'player_name': 'Dylan Coghlan',
  'player_pos': 'Defenseman'},
 {'player_id': 'ID8475215',
  'player_name': 'Robin Lehner',
  'player_pos': 'Goalie'},
 {'player_id': 'ID8476448',
  'player_name': 'William Karlsson',
  'player_pos': 'Center'},
 {'player_id': 'ID8477406',
  'player_name': 'Mattias Janmark',
  'player_pos': 'Center'},
 {'player_id': 'ID8475188',
  'player_name': 'Brayden McNabb',
  'play

In [176]:
# Boxscore JSON
# Get the number of shots, goals and icetime by game for each skater
json_object = json.loads(boxscore_json)

# first layer = the game (number)
# second layerS

arlo_swag = []
player_id = list(json_object[0]['teams']['home']['players'].keys())

for i in player_id:
    #pid = i
    player_data = json_object[0]['teams']['home']['players'][i]['person']
    arlo_swag.append(player_data)
    #arlo_swag.append(pid)


arlo_swag
df1 = pd.DataFrame.from_dict(arlo_swag)
df1['pid'] = 'ID' + df1['id'].astype(str)
df2 = pd.concat([df1.drop(['currentTeam'], axis=1), df1['currentTeam'].apply(pd.Series)], axis=1)
df2 = df2.rename(columns={"id": "team_id", "name": "team_name", "link": "team_link"})
df2 = pd.concat([df2.drop(['primaryPosition'], axis=1), df2['primaryPosition'].apply(pd.Series)], axis=1)
df2 = df2.rename(columns={"code": "pos_code", "name": "pos_name", "type": "pos_type", "abbreviation": "pos_abbr"})



df2.head(4)

,team_id,fullName,team_link,firstName,lastName,primaryNumber,birthDate,currentAge,birthCity,birthStateProvince,...,shootsCatches,rosterStatus,pid,team_id,team_name,team_link,pos_code,pos_name,pos_type,pos_abbr
0,8476905,Chandler Stephenson,/api/v1/people/8476905,Chandler,Stephenson,20,1994-04-22,27,Saskatoon,SK,...,L,Y,ID8476905,54,Vegas Golden Knights,/api/v1/teams/54,C,Center,Forward,C
1,8474149,Evgenii Dadonov,/api/v1/people/8474149,Evgenii,Dadonov,63,1989-03-12,32,Chelyabinsk,NaN,...,L,Y,ID8474149,54,Vegas Golden Knights,/api/v1/teams/54,R,Right Wing,Forward,RW
2,8476539,Jonathan Marchessault,/api/v1/people/8476539,Jonathan,Marchessault,81,1990-12-27,31,Cap-Rouge,QC,...,R,Y,ID8476539,54,Vegas Golden Knights,/api/v1/teams/54,C,Center,Forward,C
3,8475913,Mark Stone,/api/v1/people/8475913,Mark,Stone,61,1992-05-13,29,Winnipeg,MB,...,R,Y,ID8475913,54,Vegas Golden Knights,/api/v1/teams/54,R,Right Wing,Forward,RW
